# **Algorithmic Methods of Data Mining - Fall 2022**

## **Homework 4: Getting to know your customers**

**Packages that are used troughout the notebook:**

In [1]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import numpy as np
import datasets

# For progress_apply
tqdm.pandas()

datas = datasets.Datasets()

## 1. Finding Similar Costumers

### 1.1 Set up the data

Starting off, below is the head of the processed transaction dataset. Briefly, we have:
- renamed the columns to make it easier to work with,
- created customer IDs (cid) from scratch, as the original dataset was not consistent with the customer IDs. The new customer IDs are unique identifiers for the customer DOB, location and balance columns. We have made a post about this in the Slack channel,
- merged the transaction date and time into one column, and converted the date to a datetime object.

In [2]:
transactions = datas.getTransactions().copy()
transactions.head()

,tid,cid,cdob,cgender,clocation,cbalance,tdate,tamount
0,T1,0,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02 14:32:07,25.0
1,T2,1,2057-04-04,M,JHAJJAR,2270.69,2016-08-02 14:18:58,27999.0
2,T3,2,1996-11-26,F,MUMBAI,17874.44,2016-08-02 14:27:12,459.0
3,T4,3,1973-09-14,F,MUMBAI,866503.21,2016-08-02 14:27:14,2060.0
4,T5,4,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02 18:11:56,1762.5


Below is the head of the customers dataset that was obtained by grouping the transactions dataset by the customer IDs. Since we believe that the DOB, gender, location, and balance are unique for each customer, we did not have to do any aggregation when grouping the transactions dataset.

In [3]:
customers = datas.getCustomers().copy()
customers.head()

,dob,gender,location,balance
cid,,,,
0,1994-01-10,F,JAMSHEDPUR,17819.05
1,1957-04-04,M,JHAJJAR,2270.69
2,1996-11-26,F,MUMBAI,17874.44
3,1973-09-14,F,MUMBAI,866503.21
4,1988-03-24,F,NAVI MUMBAI,6714.43


We will create shingles for each customer by using their age, average transaction amount, and balance. To do this, we are going to divide these variables into 10% quantiles, and assign a quantile to each customer. Therefore, our shingle matrix will consist of $3\times10=30$ rows, and a column for each customer. The values in the matrix will be 1 if the customer belongs to the corresponding quantile, and 0 otherwise.

In [4]:
# Creating the new features
customers['age'] = ((transactions['tdate'].max() - customers['dob']).dt.days // 365).astype(int)
customers['ave_tamount'] = transactions.groupby('cid')['tamount'].mean()

# Creating quantile groups
quantiles = 10
customers['age_group'] = pd.qcut(customers['age'], quantiles, labels=False)
customers['balance_group'] = pd.qcut(customers['balance'], quantiles, labels=False)
customers['tamount_group'] = pd.qcut(customers['ave_tamount'], quantiles, labels=False)

customers.head()

,dob,gender,location,balance,age,ave_tamount,age_group,balance_group,tamount_group
cid,,,,,,,,,
0,1994-01-10,F,JAMSHEDPUR,17819.05,22,80.111111,0,5,0
1,1957-04-04,M,JHAJJAR,2270.69,59,9503.445000,9,2,9
2,1996-11-26,F,MUMBAI,17874.44,19,940.348182,0,5,5
3,1973-09-14,F,MUMBAI,866503.21,43,2537.346923,8,9,8
4,1988-03-24,F,NAVI MUMBAI,6714.43,28,1606.135000,3,3,7


We also have to store the intervals of the quantiles, since we will use them again when querying for similar customers. Futhermore we will need a function that give us the quantile group of a value, given the intervals of the quantiles.

In [5]:
# Store the quantile intervals
age_groups = pd.qcut(customers['age'], quantiles).unique().sort_values()
balance_groups = pd.qcut(customers['balance'], quantiles).unique().sort_values()
tamount_groups = pd.qcut(customers['ave_tamount'], quantiles).unique().sort_values()

# Get the group of a value
def getGroup(group, value):
    return np.where([value in g for g in group])[0][0] if len(np.where([value in g for g in group])[0]) > 0 else None

getGroup(age_groups, 29) # Test: should return 4

4

Now we are going to create our shingle matrix.

In [6]:
# make dummy variables for categorical variables
shingles = pd.DataFrame(index=customers.index)
shingles = pd.concat([shingles, pd.get_dummies(customers['age_group'], prefix='age')], axis=1)
shingles = pd.concat([shingles, pd.get_dummies(customers['balance_group'], prefix='balance')], axis=1)
shingles = pd.concat([shingles, pd.get_dummies(customers['tamount_group'], prefix='tamount')], axis=1)
# transpose to get the shingles
shingles = shingles.T
shingles.head()

cid,0,1,2,3,4,5,6,7,8,9,...,162301,162302,162303,162304,162305,162306,162307,162308,162309,162310
age_0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
age_1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
age_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
age_3,0,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
age_4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 1.2 Fingerprint hashing

In [ ]:
# import sparse matrix for shingles
from scipy import sparse

def get_signatures(shingles, signature_length: int) -> np.ndarray:
    # set seed for reproducibility
    np.random.seed(123)
    signatures = np.full((signature_length, shingles.shape[1]), np.nan, dtype=np.int32)
    for i in range(signature_length):

        idx = np.random.choice(shingles.index, shingles.shape[0], replace=False)
        sparse_shingles = sparse.csc_matrix(shingles.loc[idx].values)
        rowidx = sparse_shingles.nonzero()[0]
        colidx = sparse_shingles.nonzero()[1]
        signatures[i,:] = rowidx[np.unique(colidx, return_index=True)[1]]
    return signatures

In [ ]:
signatures = get_signatures(shingles, signature_length=20)
signatures.shape

In [ ]:
def create_buckets(signatures, band_size):
    buckets = {}
    rows = signatures.shape[0] // band_size
    for i in range(rows):
        for j in range(signatures.shape[1]):
            bucket = (i, *signatures[i*band_size:(i+1)*band_size, j])
            if bucket not in buckets:
                buckets[bucket] = []
            buckets[bucket].append(j)
    return buckets

In [ ]:
# check if equal
band_size = 2
buckets = create_buckets(signatures, band_size=2)

#### 1.3 Locality Sensitive Hashing

Now, we first parse query so that it is in the same format as the shingles.

In [ ]:
query_users = datas.getQueryUsers()
query_users['age_group'] = query_users['age'].apply(lambda x: getGroup(age_groups, x))
query_users['balance_group'] = query_users['balance'].apply(lambda x: getGroup(balance_groups, x))
query_users['tamount_group'] = query_users['tamount'].apply(lambda x: getGroup(tamount_groups, x))
query_users.index = range(len(query_users))

query_shingles = pd.DataFrame(index=query_users.index)
query_shingles = pd.concat([query_shingles, pd.get_dummies(query_users['age_group'], prefix='age')], axis=1)
query_shingles = pd.concat([query_shingles, pd.get_dummies(query_users['balance_group'], prefix='balance')], axis=1)
query_shingles = pd.concat([query_shingles, pd.get_dummies(query_users['tamount_group'], prefix='tamount')], axis=1)
query_shingles = query_shingles.T

query_signatures = get_signatures(query_shingles, signature_length=20)

query_shingles.head()

We have implemented 2 functions to query users. The first takes intersection of all the matching buckets, and the second takes union of all the matching buckets.

In [ ]:
def get_intersection(buckets, query_signatures, band_size):

    intersection = {j: set() for j in range(query_signatures.shape[1])}
    num_bucket_matches = {j: 0 for j in range(query_signatures.shape[1])}

    rows = query_signatures.shape[0] // band_size

    for i in range(rows):
        for j in range(query_signatures.shape[1]):
            bucket = (i, *query_signatures[i*band_size:(i+1)*band_size, j])
            if bucket in buckets.keys():
                intersection[j] = intersection[j] & set(buckets[bucket]) if len(intersection[j]) > 0 else set(buckets[bucket])
                num_bucket_matches[j] += 1

    return intersection, num_bucket_matches

def get_unions(buckets, query_signatures, band_size):

    unions = {j: set() for j in range(query_signatures.shape[1])}
    num_bucket_matches = {j: 0 for j in range(query_signatures.shape[1])}

    rows = query_signatures.shape[0] // band_size

    for i in range(rows):
        for j in range(query_signatures.shape[1]):
            bucket = (i, *query_signatures[i*band_size:(i+1)*band_size, j])
            if bucket in buckets.keys():
                unions[j] = unions[j].union(buckets[bucket])
                num_bucket_matches[j] += 1

    return unions, num_bucket_matches

In [ ]:
band_size = 2

buckets = create_buckets(signatures, band_size=band_size)

intersections, _ = get_intersection(buckets, query_signatures, band_size=band_size)
print('Number of matching customers by intersection for the first 10 queries:')
for i in range(10):
    print(len(intersections[i]), end='\t')


print('\nNumber of matching customers by union for the first 10 queries:')
unions, _ = get_unions(buckets, query_signatures, band_size=band_size)
for i in range(10):
    print(len(unions[i]), end='\t')


As you can see above, sometimes the intersection of the buckets is empty. In order to consistently rely on this method we must tune the number shingles and bands to make sure that the intersection is not empty. Let's inspect an example where we get a non-empty intersection.

In [ ]:
# get last query user
query_users.iloc[-1]

In [ ]:
customers.iloc[list(intersections[45])].head()

The customers above are pretty similar in their balance, transaction amount, and age. So we can say that our algorithm is working for some cases.

## 2. Grouping customers together!

### 2.1 Set up the data

#### 2.1.1 Cleaning transactions data

#### 2.1.2 Customer data

In [ ]:
customersX = datas.getCustomersX().copy()
customersX[customersX.columns[:9]].head()

So we are sparing you to look at all the data cleaning and preprocessing we did, at least in this notebook. You may find the code in datasets.py file. Rather we are going to go over the variables we created and how we created them. So as we have explaiend before we have created new customer IDs. And the data we have for these customers have unique gender, location, balance; thus the 3 of the 7 variables that you asked for is automatically implemented. We have calculated the other 4 variables which were required.

In [ ]:
customersX[customersX.columns[9:]].head()

Moving on, you can see the variables we have added to the data. We have added the following variables:
1. **age**: The age of the customer
2. **tamount_balance_ratio**: The ratio of the average transaction amount to the balance of the customer
3. **log_balance**: The log of the balance of the customer
4. **log_tamount**: The log of the average transaction amount of the customer
5. **balance_age_ratio**: The ratio of the balance to the age of the customer
6. **balance_leq_avg_salary**: Whether the balance of the customer is less than or equal to the average salary in India, that is 3200 INR.
7. **top_30_loc**: Whether the customer is from one of the top 30 most frequent locations in the dataset.
8. **tranaction_freq**: The frequency of the transactions of the customer.
9. **transaction_freq_std**: The standard deviation of the transaction frequency of the customer. This is to indicate how consistent the customer is in terms of transaction frequency.
10. **vol_weighted_transaction_freq**: The volume weighted transaction frequency of the customer. This is to indicate how much the customer spends in terms of transaction frequency.
11. **t_max**: The maximum transaction amount of the customer.
12. **t_min**: The minimum transaction amount of the customer.
13. **t_std**: The standard deviation of the transaction amount of the customer.
14. **t_skew**: The skewness of the transaction amount of the customer.
15. **t_kurt**: The kurtosis of the transaction amount of the customer.
16. **tamount_age_ratio**: The ratio of the average transaction amount to the age of the customer.
17. **boomer**: Whether the customer is a boomer or not, that is, above 60 years of age.
18. **zoomer**: Whether the customer is a zoomer or not, that is, below 20 years of age.
19. **days_till_bd**: The number of days till the customer's birthday. This can eliminate extra spending due to customer's birthday.
20. **zodiac**: The zodiac sign of the customer.

#### 2.2 Choose your features (variables)!

We will standardize our data before clustering because we want to give **equal importance** to all the variables. We will drop the location, zodiac variables because we believe it will cause overfitting. We will also convert columns with boolean values to 0 and 1.

In [ ]:
X = customersX.copy()
X.drop(['dob', 'location', 'zodiac'], axis=1, inplace=True)
X.loc[:, X.dtypes == 'bool'] = X.loc[:, X.dtypes == 'bool'].astype('int')
X['gender'] = X['gender'].apply(lambda x: 1 if x == 'M' else 0)
print('The number of non-numeric features is: ', sum([x not in ['int64', 'float64'] for x in X.dtypes]))

In [ ]:
X = (X - X.mean()) / X.std()
print('The number of cols with absolute mean larger than 0.001 is: ', sum(abs(X.mean()) > 0.001))
print('The number of cols with absolute std larger than 1.0001 or smaller than 0.9999 is: ', sum(abs(X.std() - 1) > 0.001) + sum(X.std() < 0.9999))

We will apply PCA to reduce the dimensionality of our data. Since we have standardised our data, it is also equivalent to Factor Analysis with the assumption of the noise of the components is isotropic. We also don't use any categorical variables, so we don't need to use Multiple correspondence analysis.

In [ ]:
# import pca
from sklearn.decomposition import PCA
pca = PCA().fit(X)
# Plot the explained variance
pd.Series(np.cumsum(pca.explained_variance_ratio_)).plot(
    title='Explained variance ratio by PCA components',
    figsize=(20, 4),
    grid=True,
    xlabel='PCA components',
    ylabel='Explained variance ratio'
)

print('The number of PCA components that explain 70% of the variance is: ', np.where(np.cumsum(pca.explained_variance_ratio_) > 0.70)[0][0] + 1)

In [ ]:
pca = PCA(n_components=9).fit(X)
XPC = pca.transform(X)

#### 2.3 Clustering!

In [ ]:
from pyspark import SparkContext

def assign_cluster(row, centroids):
    cluster = np.argmin(np.sum((centroids - row) ** 2, axis=1))
    return (cluster, (1, row))


def update_centroids(x, y):
    # Returns the sum of number of observations and the sum of the rows
    return (x[0] + y[0], x[1] + y[1])

def kmeans(X, k, max_iter=10, epsilon=0.0001, verbose=True):
    sc = SparkContext.getOrCreate()
    rdd = sc.parallelize(X)

    centroids = X[np.random.choice(X.shape[0], k, replace=False), :]
    sse = np.inf

    for j in (pbar:= tqdm(range(max_iter), desc='KMeans iterations')):
        mapResults = rdd.map(lambda x: assign_cluster(x, centroids)).reduceByKey(update_centroids).collect()
        centroids = np.array([mapResults[i][1][1] / mapResults[i][1][0] for i in range(k)])

        
        clusters = [row[0] for row in rdd.map(lambda x: assign_cluster(x, centroids)).collect()]
        new_sse = np.sum((np.dot(pd.get_dummies(clusters).values, centroids) - X) ** 2)
        
        if (new_sse / sse) > (1 - epsilon):
            if verbose:
                print(f'Converged, in {j + 1} iterations :))')
            break
        
        if verbose:
            pbar.set_description(f'KMeans iterations {j + 1}/{max_iter} (SSE: {sse:.2f}), rate: {(new_sse / sse):.5f}')
        sse = new_sse


    return clusters, centroids, sse

In [ ]:
number_of_clusters_grid = [2, 3, 4, 5, 6, 7, 8, 9, 10]
sse_scores = np.inf * np.ones(len(number_of_clusters_grid))
num_rep = 1

for i, k in enumerate(number_of_clusters_grid):
    for j in range(num_rep):
        _, _, sse_score = kmeans(XPC, k, 50)
        if sse_score < sse_scores[i]:
            sse_scores[i] = sse_score

pd.Series(sse_scores, index=number_of_clusters_grid).plot(
    title='SSE by number of clusters',
    figsize=(20, 4),
    grid=True,
    xlabel='Number of clusters',
    ylabel='SSE'
)

We were planning to support our decision with the elbow method, however, as you can see on the plot above, there are no clear "elbows". Practically, banks want customer segments that are easy to understand, and reflect real life. So we believe that the number of clusters should be between 3 and 7. Looking at the plot we see decreases in 4 and 6, so we will continue with these two numbers.

In [ ]:
clusters4, centroids4, _ = kmeans(XPC, 4, 50)
clusters6, centroids6, _ = kmeans(XPC, 6, 50)

In [ ]:
import matplotlib.pyplot as plt
# Plot according to the first two principal components
fig, axs = plt.subplots(1, 2, figsize=(20, 10))


# Scaterplot with 4 clusters
axs[0].scatter(XPC[:, 0], XPC[:, 1], c=clusters4, cmap='tab10', alpha=0.5)
axs[0].scatter(centroids4[:, 0], centroids4[:, 1], marker='x', s=100, c='k')
axs[0].set_title('KMeans clustering with 4 clusters')
axs[0].set_xlabel('PC1'); axs[0].set_ylabel('PC2')
axs[0].set_xlim(-5,20); axs[0].set_ylim(-20,20)
axs[0].grid()

# Scaterplot with 6 clusters
axs[1].scatter(XPC[:, 0], XPC[:, 1], c=clusters6, cmap='tab10', alpha=0.5)
axs[1].scatter(centroids6[:, 0], centroids6[:, 1], marker='x', s=100, c='k')
axs[1].set_title('KMeans clustering with 6 clusters')
axs[1].set_xlabel('PC1'); axs[1].set_ylabel('PC2')
axs[1].set_xlim(-5,20); axs[1].set_ylim(-20,20)
axs[1].grid()

plt.show()


#### 2.4 Analysing your results!

Now we select 2-3 variables, that is; gender, age, balance; to create our pivot table.

In [ ]:
selected_variables = ['age', 'gender', 'balance']
cust4pivot = customersX.copy()

cust4pivot['age_group'] = pd.qcut(customersX['age'], 4, labels=False)
cust4pivot['balance_group'] = pd.qcut(customersX['balance'], 4, labels=False)

# make dummies
cust4pivot = pd.get_dummies(cust4pivot, columns=['gender', 'age_group', 'balance_group'])

# pivot table
# gender_0	gender_1	age_group_0	age_group_1	age_group_2	age_group_3	balance_group_0	balance_group_1	balance_group_2	balance_group_3
cols = ['gender_0', 'gender_1', 'age_group_0', 'age_group_1', 'age_group_2', 'age_group_3', 'balance_group_0', 'balance_group_1', 'balance_group_2', 'balance_group_3']
cust4pivot = cust4pivot.pivot_table(index='clusters4', values=cols, aggfunc='mean')
pd.options.display.float_format = '{:,.2f}'.format

cust4pivot.columns = ['below 25', '25 to 28', '28 to 34', '34+', 'below 5240 INR', '5240 to 17230 INR', '17230 to 55530 INR', '55530+ INR', 'F', 'M']
cust4pivot = 100 * cust4pivot
cust4pivot

Here is our indepth analysis of the clusters:

In [ ]:
variables_to_inspect = ['age', 'gender', 'balance', 'log_balance', 'avg_tamount', 'top_30_loc', 'transaction_freq']
variables_to_inspect = [*customersX.columns[1:2], *customersX.columns[4:25]]
customersX['clusters4'] = clusters4
customersX['clusters6'] = clusters6
customersX['gender'] = customersX['gender'].apply(lambda x: 1 if x == 'M' else 0)

c4stomers = customersX.groupby('clusters4').agg({col: 'mean' for col in variables_to_inspect})
(c4stomers - customersX[variables_to_inspect].mean()) / customersX[variables_to_inspect].std()
# merge with columns
Z_scores = (c4stomers - customersX[variables_to_inspect].mean()) / customersX[variables_to_inspect].std()

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
# Bar plot
def colors_from_values(values, palette_name):
    # normalize the values to range [0, 1]
    normalized = values
    normalized[normalized < 0] = - normalized[normalized < 0] / normalized.min() * 500
    normalized[normalized > 0] = normalized[normalized > 0] / normalized.max() * 500 -1
    normalized = normalized + 500
    # convert to indices
    indices = np.round(normalized).astype(np.int32)
    # use the indices to get the colors
    palette = sns.color_palette(palette_name, 1000)
    return np.array(palette).take(indices, axis=0)

fig, axs = plt.subplots(2, 2, figsize=(20, 10))

fig.suptitle('Z-scores for each cluster', fontsize=16)

# 1st cluster bar plot
sns.barplot(x=Z_scores.iloc[0, :].sort_values(ascending=False), y=Z_scores.iloc[0, :].sort_values(ascending=False).index,
ax=axs[0, 0], palette=colors_from_values(Z_scores.iloc[0, :].sort_values(ascending=False), 'Spectral'), orient = 'h')
axs[0, 0].set_title('Irregular spenders')

# 2nd cluster bar plot
sns.barplot(x=Z_scores.iloc[1, :].sort_values(ascending=False), y=Z_scores.iloc[1, :].sort_values(ascending=False).index,
ax=axs[0, 1], palette=colors_from_values(Z_scores.iloc[1, :].sort_values(ascending=False), 'Spectral'), orient = 'h')
axs[0, 1].set_title('Poor folk')

# 3rd cluster bar plot
sns.barplot(x=Z_scores.iloc[2, :].sort_values(ascending=False), y=Z_scores.iloc[2, :].sort_values(ascending=False).index,
ax=axs[1, 0], palette=colors_from_values(Z_scores.iloc[2, :].sort_values(ascending=False), 'Spectral'), orient = 'h')
axs[1, 0].set_title('Conservatives')

# 4th cluster bar plot
sns.barplot(x=Z_scores.iloc[3, :].sort_values(ascending=False), y=Z_scores.iloc[3, :].sort_values(ascending=False).index,
ax=axs[1, 1], palette=colors_from_values(Z_scores.iloc[3, :].sort_values(ascending=False), 'Spectral'), orient = 'h')
axs[1, 1].set_title('Rich folk')

plt.show()

From the pivot table we get a general idea of clusters. However, we thought it did not unravel the complete story. So we decided to look at the clusters features in terms of their distance from to overall means. In the plot above, you can see the distance of each cluster from the overall mean. The features that have distances with high magnitude characterize their clusters. So here is our interpretation of the clusters:

**Cluster 1**: This cluster is characterized by the fact that the customers have dispersed spending habits, because their kurtosis and skewness for their transaction amounts are high compared to the rest of dataset. Their minimum transactions are very low and maximum are very high, which means they use their cards for any kind of spending. As a bank we would like identify these customers of being more adapted to the digital world, and offer them more digital services.

**Cluster 2**: This cluster is the poor people. They have balances lower than the average salary. This means that they are more likely default on their payments. As a bank we would be more careful with these customers, and offer them more services that would help them manage their finances better.

**Cluster 3**: We can label this cluster as the "conservatives" or "traditionalists". They have low number of transactions, and relatively old. They only use their cards to pay very high amounts. This means that they do not use their cards for small purchases. As a bank we would target them as customers with more potential to spend, and try to educate them on the benefits of using their cards for small purchases.

**Cluster 4**: Eat the rich. This cluster is characterized by the fact that they have very high balances, and very high transaction amounts. They are also very old and boomer. We would offer them personalized services, and try to make them feel special. For example, lounge access, and private transfers etc.

### Bonus

In [ ]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering

dbscan = DBSCAN(eps=0.3, min_samples=10).fit(XPC[:, :2])

plt.figure(figsize=(10, 10))

plt.scatter(XPC[:, 0], XPC[:, 1], c=dbscan.labels_, cmap='tab10', alpha=0.5)
plt.title('DBSCAN clustering')

plt.xlabel('PC1'); plt.ylabel('PC2')
plt.xlim(-5,20); plt.ylim(-20,20)
plt.show()

Our data is very dense, in the center and sparse in the edges. This is because, economically, the majority of the population is in the low-middle class, and the upper class is very small. As a result, DBSCAN groups all low and middle class customers together, and leaves the upper class customers alone. The distance between the middle and lower class is very small compared to the distance between the upper class and the middle class. Therefore density is not a good measure and **DBSCAN** is not a good clustering method for our data.

In [ ]:
hierarchical = AgglomerativeClustering(
    n_clusters=None, 
    linkage='ward',
    distance_threshold = 7
).fit(XPC)

In [ ]:
# import dendrogram
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count +=1
            else:
                current_count += counts[child_idx-n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                     counts]).astype(float)
    
    dendrogram(linkage_matrix, **kwargs)

fig, ax = plt.subplots(figsize = (20,10))
ax.set_title("Hierarchical clustering dendrogram")
#plot the top three levels of the dendrogram
plot_dendrogram(hierarchical, truncate_mode='level', p=7)
ax.set_xlabel("Number of points in node")
plt.show()

# This code is inspired from Philip Wilkinson's post on Towards Data Science

In [ ]:
# hierarchical clustering with 4 clusters
hierarchical4 = AgglomerativeClustering(
    n_clusters=4,
    linkage='ward',
    distance_threshold = None
).fit(XPC)

# get the cluster labels
hierarchical4.labels_

In [ ]:
customersX['hierarchical4'] = hierarchical4.labels_
customers_h4 = customersX.groupby('hierarchical4').agg({col: 'mean' for col in variables_to_inspect})
Z_h4 = (customers_h4 - customersX[variables_to_inspect].mean()) / customersX[variables_to_inspect].std()

for i in range(4):
    print(f'Top explanatory features for Cluster {i+1}')
    t = Z_h4.loc[i, Z_h4.iloc[i, :].abs().sort_values(ascending=False)[:5].index].sort_values(ascending=False)
    for j in range(len(t)):
        print(f'{j+1}. {t.index[j]}: \t \t {t.iloc[j].round(2)}')
    

For hierarchical clustering, we have decided to inspect 4 clusters. One immediate observation is that the some clusters are way more marginal compared to the results we got in K-means. As you can see, cluster 1 and 4 has features that are very distant from the mean, which is a sign of being even more charactestic. For example, cluster 4 are people with very high balance and age. In our previous results we have also identified a segment of rich customers, however, the results we get from hierarchical clustering are more extreme. We can further our analysis looking into more clusters, that is, dividing up the current clusters into more clusters. And doing so we can better understand the segments of customers. We think this is a good method to use for a bank to understand their customers, where computational power is not a concern.

In [ ]:
# import kmeans from sklearn
from sklearn.cluster import KMeans


n_clusters = 3
kmeans_sk = KMeans(n_clusters=n_clusters)
rows = 5
fig, axs = plt.subplots(rows, 2, figsize=(20, 25))

for i in range(rows):
    kmeans_sk.fit(XPC[:, :2])
    axs[i, 0].scatter(XPC[:, 0], XPC[:, 1], c=kmeans_sk.labels_, cmap='tab10', alpha=0.5)
    axs[i, 0].set_title(f'K-means clustering (sklearn) {i+1}')
    axs[i, 0].set_xlabel('PC1'); axs[i, 0].set_ylabel('PC2')
    axs[i, 0].set_xlim(-5,20); axs[i, 0].set_ylim(-20,20)

    clusters, centroids, _ = kmeans(XPC[:, :2], k=n_clusters, max_iter=100, verbose=False)
    axs[i, 1].scatter(XPC[:, 0], XPC[:, 1], c=clusters, cmap='tab10', alpha=0.5)
    axs[i, 1].set_title(f'K-means clustering (our code) {i+1}')
    axs[i, 1].set_xlabel('PC1'); axs[i, 1].set_ylabel('PC2')
    axs[i, 1].set_xlim(-5,20); axs[i, 1].set_ylim(-20,20)

plt.show()



In our code we initialize randomly and the module we have imported from sklearn initialize with KMeans++. In the plot above, you can observe that the module we have imported from sklearn always has only 2 large clusters, and the third is not even visible. On the other hand, in some of the results that we get from our code has 3 balanced clusters. This is due to the nature of clustering processes. Clustering, in the end, has qualitative implications, and sometimes quantitative metrics do not give a good picture of the results. Surprisingly, our code does a better job in having more balanced clusters, if that is what you are looking for.

#### Summary of clustering comparison

All in all, we have learned that not every clustering algorithm fits every dataset. In our case, DBSCAN is not a good choice because of the distribution of our data, compared to K-means and hierarchical clustering. Moreover, the motivation behind the clustering is also a factor that should be considered. For example, if you want to have a balanced number of clusters, in our case, K-means with random initiliziation produced better results. Finally, we think that if our goal is to do qualitative analysis, then we should use hierarchical clustering, since it provides sub-clusters, and we can further our analysis by looking into the sub-clusters.

### Command Line Question

In [ ]:
%%bash

echo '1-The location that has the maximum number of purchases been made:'

awk -F, '{print($5)}' data/bank_transactions.csv | sort | uniq -c | sort -nr | head -1

echo '2-The most spending purchases are: '

awk -F, '{$4+=$9}END{ if(list["F"] > list["M"]){ print("F");} else{ print("M");}}' data/bank_transactions.csv

echo '3-The customer with the highest average transaction amount: '

awk -F, 'BEGIN{FS=","} {if(NR>1){arr[$2]= arr[$2]+$9; count[$2]++}} END{for(c in arr){arr[c]= arr[c]/count[c]} asort(arr, sortedarr);for(l in arr){if(arr[l]==sortedarr[length(sortedarr)])print arr[l], l}}' bank_transactions.csv

### Algorithmic Question

In [ ]:
def check_enough_guards(N: int, M: int, G: int, arriving: list):
    """
    Check if there are enough guards to protect the gates
    """
    enough  = True # do we have enough guards?
    history = {}   # history of first and last time a student visited a gate

    for i, student in enumerate(arriving):

        if student not in history.keys():
            history[student] = [i, -1] # first time, last time
        else:
            history[student][1] = i    # update last time
        

    for i in range(N):
        intersections = 0

        for gate in history.keys():
            if history[gate][0] <= i <= history[gate][1]:
                intersections += 1
        
        if intersections > G:
            enough = False
            break

    print('YES') if enough else print('NO')
    # print(history, interactions)

## Case 1
arriving = [1, 1, 3, 3, 3]
#arriving = [3, 3, 1, 1, 3, 3, 2, 2, 3, 3, 2, 2,]

N, M, G = 4, len(arriving), 1

print(f'N = {N}, M = {M}, G = {G}, arriving = {arriving}')
check_enough_guards(N, M, G, arriving)


# Case 2
arriving = [1, 2, 1, 2, 2]
N, M, G = 2, len(arriving), 1

print(f'N = {N}, M = {M}, G = {G}, arriving = {arriving}')
check_enough_guards(N, M, G, arriving)